# 📓 The GenAI Revolution Cookbook

**Title:** How to Build a Model Context Protocol (MCP) Server in Python

**Description:** Learn how to build an MCP server in Python to standardize and reuse AI tools, resources, and prompts across applications. This hands-on guide walks you through server setup, client testing, and GPT-4 chatbot integration for production-ready systems.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Introduction

The Model Context Protocol (MCP) lets you define tools, resources, and prompts once and expose them to any MCP-capable client—from CLIs to agents to chatbots. Instead of rewriting tool logic for each application, you build a single server that clients discover and invoke automatically.

This guide walks you through building a minimal Python MCP server over stdio, testing it with a client, and verifying that tools, resources, and prompts work as expected. By the end, you'll have a working server that exposes math tools, static documentation, and a prompt template—all runnable in a notebook or local environment.

For a comprehensive overview of how MCP standardizes tool and data access, see our [Model Context Protocol (MCP) Explained [2025 Guide for Builders]](/article/model-context-protocol-mcp-explained-2025-guide-for-builders).

## Why Use MCP for This Problem

When you need to share tools across multiple applications, you have several options:

- **Shared Python package**: Requires every app to import and maintain the same library version; no runtime discovery or schema negotiation.
- **Bespoke HTTP microservice**: Adds network overhead, requires custom API design, and lacks standardized tool metadata.
- **OpenAI-native tools defined per app**: Forces you to duplicate tool definitions in every codebase; no single source of truth.
- **Agent-framework-specific tools**: Locks you into one framework's API; porting to another requires rewriting.

MCP solves these problems by providing:

- **Automatic discovery**: Clients list available tools, resources, and prompts at runtime.
- **Standardized schemas**: JSON Schema definitions ensure consistent validation and documentation.
- **Transport abstraction**: Stdio, SSE, or WebSocket—clients and servers negotiate capabilities without custom protocols.

Centralizing these capabilities in one server removes repetition and lets clients discover and invoke standardized functionality automatically. If you want to avoid subtle bugs caused by tokenization quirks, check out our guide on [Tokenization Pitfalls: Invisible Characters That Break Prompts and RAG](/article/tokenization-pitfalls-invisible-characters-that-break-prompts-and-rag-2).

A single MCP server means one place to update logic and metadata, one schema for validation, and consistent behavior across apps. However, remember that LLM context is not infinite memory—if you're scaling up prompt sizes or chaining calls, you should be aware of [Context Rot - Why LLMs "Forget" as Their Memory Grows](/article/context-rot-why-llms-forget-as-their-memory-grows-3).

## Core Concepts for This Use Case

Before building, understand these MCP primitives:

- **Tools**: Functions the client can invoke with typed arguments; the server executes and returns results.
- **Resources**: Static or dynamic data (text, JSON, binary) identified by URI; clients read them on demand.
- **Prompts**: Reusable templates that generate messages for LLM conversations; clients fetch and render them with arguments.
- **Stdio transport**: Server and client communicate over standard input/output; simplest option for local development and subprocess integration.
- **Schema negotiation**: Server advertises tool input schemas (JSON Schema) and resource MIME types; clients validate and adapt accordingly.

## Setup

Install the required packages in a notebook cell or terminal:

In [ ]:
!pip install "mcp[cli]>=0.9.0" anyio>=4.0.0 openai>=1.40.0

This installs the MCP SDK, async runtime, and OpenAI client. Pin versions for reproducibility.

## Using the Tool in Practice

### Build the MCP Server

Create a server that exposes two math tools, a static markdown resource, and a prompt template. This example uses stdio transport for simplicity.

Write the server to a file using a notebook cell:

In [ ]:
%%writefile mcp_server.py
# Purpose: Minimal MCP server exposing math tools, a static resource, and a prompt template over stdio.

import anyio
from typing import Annotated

# MCP server APIs
from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import (
    PromptMessage,
    TextResourceContents,
    Resource,
    Prompt,
)

# Instantiate the MCP server with a unique name
server = Server("calc-server")

@server.tool()
async def add(a: Annotated[int, "First integer"], b: Annotated[int, "Second integer"]) -> int:
    """
    Add two integers and return the sum.

    Args:
        a (int): First integer.
        b (int): Second integer.

    Returns:
        int: The sum of a and b.
    """
    # Simple addition; no edge cases for int
    return a + b

@server.tool()
async def subtract(a: Annotated[int, "Minuend"], b: Annotated[int, "Subtrahend"]) -> int:
    """
    Subtract b from a and return the difference.

    Args:
        a (int): Minuend.
        b (int): Subtrahend.

    Returns:
        int: The result of a - b.
    """
    # Simple subtraction; no edge cases for int
    return a - b

# Resource: static markdown documentation
DOCS_ID = "docs/usage"
DOCS_CONTENT = """# Calc Server Usage

Tools:
- add(a: int, b: int): returns a + b
- subtract(a: int, b: int): returns a - b

Prompt:
- math_helper(expression: string): step-by-step computation
"""

@server.resource(DOCS_ID, mime_type="text/markdown")
async def read_docs() -> TextResourceContents:
    """
    Return static markdown documentation for the server.

    Returns:
        TextResourceContents: Markdown-formatted usage documentation.
    """
    # TextResourceContents includes text and optional annotations
    return TextResourceContents(text=DOCS_CONTENT)

@server.prompt("math_helper")
async def math_helper(expression: Annotated[str, "Math expression to compute"]):
    """
    Generate a prompt for step-by-step math computation.

    Args:
        expression (str): Math expression to compute.

    Returns:
        list[PromptMessage]: System and user prompt messages.
    """
    # System prompt instructs the assistant to show work
    system = PromptMessage(role="system", content="You are a careful math assistant. Show your work.")
    # User prompt includes the expression to compute
    user = PromptMessage(role="user", content=f"Compute the following expression step by step: {expression}")
    return [system, user]

async def main():
    """
    Main entry point: runs the MCP server over stdio until EOF.
    """
    # stdio transport: run until EOF
    async with stdio_server() as (read_stream, write_stream):
        await server.run(read_stream, write_stream)

if __name__ == "__main__":
    anyio.run(main)

The `@server.tool()` decorator registers async functions as tools. The `@server.resource()` decorator exposes static or dynamic data by URI. The `@server.prompt()` decorator defines reusable prompt templates. The server runs over stdio, reading requests from stdin and writing responses to stdout.

### Test the Server with a Client

Write a client that connects to the server, lists capabilities, calls a tool, reads a resource, and fetches a prompt:

In [ ]:
%%writefile mcp_client.py
# Purpose: Async MCP client to test server tools, resources, and prompts over stdio.

import anyio
from mcp.client.stdio import stdio_client
from mcp.client.session import ClientSession

async def main():
    """
    Connects to the MCP server, lists tools/resources/prompts, calls a tool, and fetches a prompt.

    Raises:
        Exception: If server connection or calls fail.
    """
    # Launch the server as a subprocess and connect over stdio
    async with stdio_client(["python", "mcp_server.py"]) as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            # List tools and print their names
            tools = await session.list_tools()
            print("Tools:", [t.name for t in tools.tools])

            # List resources and print their URIs
            resources = await session.list_resources()
            print("Resources:", [r.uri for r in resources.resources])

            # Read and print documentation resource if present
            for r in resources.resources:
                if r.uri.endswith("docs/usage"):
                    content = await session.read_resource(r.uri)
                    # content.contents is a list of typed chunks (e.g., text, blob)
                    for c in content.contents:
                        if hasattr(c, "text"):
                            print("Docs:\n", c.text)

            # Call 'add' tool with arguments and print the result
            result = await session.call_tool("add", {"a": 5, "b": 7})
            # result.content is list of output messages; pick first text
            out = result.content[0].text if result.content else None
            print("add(5,7) =", out)

            # Fetch prompt template and print its messages
            prompts = await session.list_prompts()
            print("Prompts:", [p.name for p in prompts.prompts])
            prompt = await session.get_prompt("math_helper", {"expression": "3*(4+2)"})
            print("Prompt messages:", [(m.role, m.content) for m in prompt.messages])

if __name__ == "__main__":
    anyio.run(main)

The client launches the server as a subprocess, establishes a session over stdio, and exercises all three primitives: tools, resources, and prompts. The `stdio_client` context manager handles process lifecycle and stream wiring.

## Run and Evaluate

Run the client to verify the server works end-to-end:

In [ ]:
!python mcp_client.py

Expected output:

In [ ]:
Tools: ['add', 'subtract']
Resources: ['docs/usage']
Docs:
 # Calc Server Usage

Tools:
- add(a: int, b: int): returns a + b
- subtract(a: int, b: int): returns a - b

Prompt:
- math_helper(expression: string): step-by-step computation

add(5,7) = 12
Prompts: ['math_helper']
Prompt messages: [('system', 'You are a careful math assistant. Show your work.'), ('user', 'Compute the following expression step by step: 3*(4+2)')]

If you see this output, your server is correctly exposing tools, resources, and prompts. The client successfully discovered and invoked each capability.

### Optional: Add Resilience to Tool Calls

For production use, wrap tool calls in a timeout and error handler. Create a helper function:

In [ ]:
%%writefile helpers/safe_call.py
# Purpose: Utility for safe, timeout-guarded MCP tool calls.

from anyio import fail_after, WouldBlock
import logging

async def safe_call_tool(mcp, name, args, seconds=10):
    """
    Call an MCP tool with a timeout and error handling.

    Args:
        mcp: MCP client session.
        name (str): Tool name.
        args (dict): Tool arguments.
        seconds (int): Timeout in seconds.

    Returns:
        Tool call result or None if failed/timed out.

    Raises:
        None: All exceptions are caught and logged.
    """
    try:
        with fail_after(seconds):
            return await mcp.call_tool(name, args)
    except WouldBlock:
        # Timeout occurred
        logging.warning(f"Tool call to {name} timed out after {seconds}s.")
        return None
    except Exception as e:
        # Log error and return None
        logging.error(f"Tool call {name} failed: {e}")
        return None

Use `safe_call_tool` in place of direct `session.call_tool` to prevent hanging or crashing on slow or failing tools.

## Conclusion

You've built a minimal MCP server that exposes tools, resources, and prompts over stdio, and verified it with a test client. This pattern lets you define capabilities once and reuse them across any MCP-compatible application—from CLIs to agents to chatbots.

### Next Steps

- **Integrate with OpenAI function calling**: Convert MCP tools to OpenAI tool schemas and route calls from GPT-4 to your server (covered in a separate guide).
- **Add dynamic resources**: Serve real-time data (e.g., database queries, API responses) instead of static markdown.
- **Explore other transports**: Use SSE or WebSocket for remote or browser-based clients.
- **Package and deploy**: Containerize your server and expose it via a network transport for multi-user access.